In [40]:
import pandas as pd
import pickle
from sklearn.impute import KNNImputer
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.impute import KNNImputer


In [41]:
motor=pd.read_csv('motor_final.csv')
non_motor=pd.read_csv('non_motor_final.csv')

In [42]:
merged = motor.merge(non_motor, how='outer', on=['PATNO'])

In [43]:
merged.columns 

Index(['PATNO', 'NP1RTOT', 'NP1PTOT', 'NP2PTOT', 'PDTRTMNT', 'NP3TOT',
       'DYSKPRES', 'NHY', 'NP4TOT', 'MSEADLG', 'COHORT_x', 'ENROLL_AGE_x',
       'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT', 'REMSLEEP_TOT',
       'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E', 'MoCA_score',
       'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC', 'HVLT_RETENTION',
       'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy', 'BJLOT_TOT',
       'COHORT'],
      dtype='object')

In [44]:
merged.drop('COHORT_x',axis=1,inplace=True)

In [45]:
merged.to_csv('merged.csv')

In [46]:
merged.isnull().sum()

PATNO                0
NP1RTOT            384
NP1PTOT            384
NP2PTOT            384
PDTRTMNT           384
NP3TOT             384
DYSKPRES           384
NHY                384
NP4TOT             384
MSEADLG            384
ENROLL_AGE_x       384
SDMTOTAL             2
STAI_TOT             2
SFT_TOT              2
SCOPA_AUT_TOT        2
REMSLEEP_TOT         2
QUIP_A               2
QUIP_B               2
QUIP_C               2
QUIP_D               2
QUIP_E               2
MoCA_score           2
LNS_TOT              2
HVLT_TOT_Recall      2
HVLT_DCR_REC         2
HVLT_RETENTION       2
GDS_TOT              2
GDS_Depressed        2
ESS_TOT              2
ESS_Sleepy           2
BJLOT_TOT            2
COHORT               2
dtype: int64

In [47]:
for cohort_value in merged['COHORT'].unique():
    # Filter the DataFrame based on cohort value
    cohort_df = merged[merged['COHORT'] == cohort_value]
    
    # Save the filtered DataFrame to a new CSV file
    cohort_df.to_csv(f'Merged_COHORT_{cohort_value}_data.csv',index=False)

In [48]:
df1 = pd.read_csv('Merged_COHORT_0.0_data.csv')
df2 = pd.read_csv('Merged_COHORT_1.0_data.csv')
df3 = pd.read_csv('Merged_COHORT_2.0_data.csv')
df4 = pd.read_csv('Merged_COHORT_3.0_data.csv')

In [49]:
# Define a function to fill missing values and apply KNN imputation
def fill_missing_and_impute(df):
    # Group DataFrame by 'PATNO' and fill missing values with median within each group
    grouped_df = df.groupby('PATNO').transform(lambda x: x.fillna(x.median()))

    # Add 'PATNO' column back to the filled DataFrame
    df_filled = pd.concat([df['PATNO'], grouped_df], axis=1)

    # Calculate the mean of each feature after filling missing values
    grouped_mean_df = df_filled.groupby('PATNO').mean().reset_index()

    # Apply KNN imputation to handle remaining missing values
    knn = KNNImputer()
    grouped_mean_df = pd.DataFrame(knn.fit_transform(grouped_mean_df), columns=grouped_mean_df.columns)
    
    return grouped_mean_df

# Apply the function to all four dataframes
df1_filled = fill_missing_and_impute(df1)
df2_filled = fill_missing_and_impute(df2)
df3_filled = fill_missing_and_impute(df3)
df4_filled = fill_missing_and_impute(df4)

In [50]:
f = pd.concat([df1_filled
               ,df2_filled,df3_filled,df4_filled])

In [51]:
f.sample(10)

,PATNO,NP1RTOT,NP1PTOT,NP2PTOT,PDTRTMNT,NP3TOT,DYSKPRES,NHY,NP4TOT,MSEADLG,...,LNS_TOT,HVLT_TOT_Recall,HVLT_DCR_REC,HVLT_RETENTION,GDS_TOT,GDS_Depressed,ESS_TOT,ESS_Sleepy,BJLOT_TOT,COHORT
1534,239947.0,1.000000,10.000000,4.000000,0.00,19.000000,0.000000,2.0000,0.000000,95.000000,...,10.0,21.0,9.0,0.0,3.0,0.0,9.0,0.0,15.0,0.0
119,3480.0,0.600000,8.600000,0.600000,0.00,3.800000,0.000000,0.0000,0.000000,99.285714,...,10.0,28.0,11.0,0.0,2.0,0.0,2.0,0.0,14.0,1.0
1602,244064.0,1.000000,7.000000,1.000000,0.00,7.000000,0.000000,0.0000,0.000000,100.000000,...,13.0,23.0,10.0,0.0,6.0,1.0,6.0,0.0,15.0,3.0
1684,292893.0,0.000000,5.000000,0.000000,0.00,2.000000,0.000000,0.0000,0.000000,100.000000,...,10.0,31.0,12.0,1.0,0.0,0.0,10.0,1.0,11.0,3.0
303,160821.0,0.666667,2.333333,0.000000,0.00,1.666667,0.000000,0.0000,0.000000,100.000000,...,10.0,27.0,11.0,0.0,1.0,0.0,2.0,0.0,8.0,1.0
1735,309923.0,1.000000,9.000000,0.000000,0.00,1.000000,0.000000,0.0000,0.000000,95.000000,...,15.0,20.0,12.0,0.0,2.0,0.0,9.0,0.0,14.0,3.0
35,3086.0,3.615385,5.923077,5.346154,0.76,14.565217,0.120000,1.6000,5.550000,89.423077,...,8.0,27.0,11.0,0.0,5.0,0.0,6.0,0.0,13.0,0.0
325,57277.0,0.642857,1.571429,0.071429,0.00,0.142857,0.071429,0.0000,0.000000,98.928571,...,9.0,23.0,10.0,0.0,1.0,0.0,4.0,0.0,12.0,3.0
278,54309.0,2.187500,13.062500,4.500000,0.00,5.187500,0.000000,0.0625,1.268182,99.062500,...,8.0,18.0,8.0,0.0,5.0,0.0,13.0,0.0,12.0,3.0
789,135121.0,0.500000,1.000000,0.000000,0.00,15.000000,0.000000,0.0000,0.000000,100.000000,...,12.0,22.0,10.0,0.0,0.0,0.0,5.0,0.0,13.0,3.0


In [52]:
f.isnull().sum()

PATNO              0
NP1RTOT            0
NP1PTOT            0
NP2PTOT            0
PDTRTMNT           0
NP3TOT             0
DYSKPRES           0
NHY                0
NP4TOT             0
MSEADLG            0
ENROLL_AGE_x       0
SDMTOTAL           0
STAI_TOT           0
SFT_TOT            0
SCOPA_AUT_TOT      0
REMSLEEP_TOT       0
QUIP_A             0
QUIP_B             0
QUIP_C             0
QUIP_D             0
QUIP_E             0
MoCA_score         0
LNS_TOT            0
HVLT_TOT_Recall    0
HVLT_DCR_REC       0
HVLT_RETENTION     0
GDS_TOT            0
GDS_Depressed      0
ESS_TOT            0
ESS_Sleepy         0
BJLOT_TOT          0
COHORT             0
dtype: int64

In [53]:
f['COHORT'].value_counts()

COHORT
0.0    1752
3.0    1737
1.0     357
2.0      81
Name: count, dtype: int64

In [1]:
X =  f.drop(['COHORT','PATNO'],axis=1)
y= f['COHORT']

NameError: name 'f' is not defined

In [ ]:
f.to_csv('merged_mnm.csv')

In [ ]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_over,y_over , 
                                   random_state=104,  
                                   test_size=0.25,  
                                   shuffle=True)

In [ ]:
svm = SVC(kernel="rbf", gamma="auto",probability=True)
svm.fit(X_train,y_train)
prediction = svm.predict(X_test)

accuracy_SVM = accuracy_score(prediction, y_test)
print(accuracy_SVM)

0.6292857142857143


In [ ]:
dec = DecisionTreeClassifier(random_state=4)
dec.fit(X_train,y_train)
pred = dec.predict(X_test)
accuracy_DEC = accuracy_score(pred, y_test)
print(accuracy_DEC)

0.935


In [ ]:
rand = RandomForestClassifier(max_depth=15, random_state=8)
rand.fit(X_train,y_train)
rand_pred = rand.predict(X_test)
rand_accuracy = accuracy_score(rand_pred,y_test)
print(rand_accuracy)

0.9507142857142857


In [ ]:
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
cross_val_results = cross_val_score(rand, X, y, cv=kf)
print(f'Cross-Validation Results (Accuracy): {cross_val_results}')
print(f'Mean Accuracy: {cross_val_results.mean()}')

Cross-Validation Results (Accuracy): [0.9389313  0.93638677 0.92229299 0.92484076 0.94522293]
Mean Accuracy: 0.9335349508111701


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Assuming X_train, X_test, y_train, and y_test are already defined

# Initialize and train the gradient boosting classifier
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)

# Make predictions on the test set
gb_pred = gb_clf.predict(X_test)

# Calculate accuracy
gb_accuracy = accuracy_score(gb_pred, y_test)
print("Gradient Boosting Classifier Accuracy:", gb_accuracy)

Gradient Boosting Classifier Accuracy: 0.9507142857142857


In [ ]:
from sklearn.ensemble import VotingClassifier
# Create a Soft Voting Classifier
labels = ['Decision ', 'Random Forest','GB']
voting_clf_hard = VotingClassifier(
	estimators=[
        (labels[0], dec),
		(labels[1], rand), # Include the second classifier (Random Forest)
		(labels[2],gb_clf ), # Include the third classifier (Naive Bayes)
	],
	voting='soft' # Specify soft voting, where class probabilities are combined
)

In [ ]:
voting_clf_hard.fit(X_train,y_train)
V_soft_pred = voting_clf_hard.predict(X_test)
voting_soft_accuracy = accuracy_score(V_soft_pred,y_test)
print(voting_soft_accuracy)

0.9571428571428572


In [ ]:
import pickle
import joblib
model=rand

with open("model_Merged.pkl", "wb") as f:
    pickle.dump(model, f)
    
with open("model_Merged.pkl", "rb") as f:
    model = pickle.load(f)
    
joblib.dump(model, "model_Merged.joblib")
model = joblib.load("model_Merged.joblib")